In [7]:
%load_ext autoreload
%autoreload 2

import os 
import sys
sys.path.append('..')
sys.path.append('../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.metrics import matthews_corrcoef

import category_encoders as ce

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from category_encoders import TargetEncoder, CatBoostEncoder


import gc


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
data_path = f'../../data/'
train_path = data_path + 'train.csv'
test_path = data_path + 'test.csv'
submission_path = data_path + 'sample_submission.csv'   

In [9]:
train = pd.read_csv(train_path)

In [5]:
# Handle missing values in the training set
missing_values_intrain = train.isnull().mean() * 100
cols_drop = missing_values_intrain[missing_values_intrain > 95].index
ds = train.drop(columns=cols_drop)

###
# Select categorical columns for encoding
cols_train = ds.select_dtypes(include=['object']).columns
cols_train = cols_train[cols_train != 'class']

# Encode categorical features
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ds[cols_train] = ordinal_encoder.fit_transform(ds[cols_train].astype(str))
ds[cols_train] = ds[cols_train].astype(int)
###

# Drop the 'id' column
ds = ds.drop(columns=['id'])

# Encode target labels
le = LabelEncoder()
ds['class'] = le.fit_transform(ds['class'])

# Split features and target
X = ds.drop(columns=['class'])
y = ds['class']


In [6]:
X

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,8.80,53,73,73,8,44,28,60,4.51,15.39,27,48,56,15,15,5,18,21,25,0
1,4.51,72,56,65,8,44,28,46,4.79,6.48,27,59,48,15,15,19,40,21,25,3
2,6.94,53,73,49,8,76,28,60,6.85,9.93,27,52,46,15,15,5,18,21,36,3
3,3.88,53,82,57,8,71,41,37,4.16,6.53,27,48,56,15,15,5,18,21,25,2
4,5.85,72,65,75,8,47,41,60,3.37,8.36,27,48,56,15,15,5,18,21,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,9.29,53,68,63,21,65,41,60,12.14,18.81,15,48,56,20,22,19,19,21,25,2
3116941,10.88,68,68,75,21,47,28,49,6.65,26.97,27,48,56,15,15,5,18,21,25,2
3116942,7.82,72,53,55,8,44,41,60,9.51,11.06,27,48,58,15,22,19,40,21,25,0
3116943,9.45,65,59,63,21,52,41,49,9.13,17.77,27,59,56,15,15,19,28,21,25,2


# XGB with hyperopt

In [12]:
!pip install hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the MCC metric for evaluation within the hyperopt objective
def mcc_metric(y_pred, dmatrix):
    y_true = dmatrix.get_label()
    y_pred = (y_pred > 0.5).astype(int)
    mcc = matthews_corrcoef(y_true, y_pred)
    return mcc

# Define the objective function for Hyperopt
def objective(params):
    model = XGBClassifier(
        alpha=params['alpha'],                    
        subsample=params['subsample'],     
        colsample_bytree=params['colsample_bytree'],  
        objective='binary:logistic',
        max_depth=int(params['max_depth']),             
        min_child_weight=params['min_child_weight'],         
        gamma=params['gamma'],                
        random_state=42,                 
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate']
    )
    
    model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_test, y_test)],
        eval_metric=lambda y_pred, dmatrix: [('mcc', mcc_metric(y_pred, dmatrix))],
        verbose=False,
        early_stopping_rounds=10
    )

    y_pred = model.predict(X_test)
    score = matthews_corrcoef(y_test, y_pred)
    return {'loss': -score, 'status': STATUS_OK}

# Define the hyperparameter search space
space = {
    'alpha': hp.loguniform('alpha', -5, 0),  # equivalent to alpha in range (exp(-5), exp(0))
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'max_depth': scope.int(hp.quniform('max_depth', 3, 15, 1)),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'gamma': hp.loguniform('gamma', -8, 0),  # equivalent to gamma in range (exp(-8), exp(0))
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 10)),
    'learning_rate': hp.loguniform('learning_rate', -5, 0)  # equivalent to learning_rate in range (exp(-5), exp(0))
}

# Run the optimization process
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)  # Ensure reproducibility
)

import pickle
with open('best_params_xgb.pkl', 'wb') as f:
    pickle.dump(best_params, f)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



  2%|▏         | 1/50 [00:17<14:06, 17.27s/trial, best loss: -0.8701685725683594]

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



  4%|▍         | 2/50 [00:25<09:33, 11.95s/trial, best loss: -0.8701685725683594]

...

 96%|█████████▌| 48/50 [12:55<00:32, 16.13s/trial, best loss: -0.9494162658031088]

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



 98%|█████████▊| 49/50 [13:15<00:17, 17.35s/trial, best loss: -0.9494162658031088]

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



100%|██████████| 50/50 [13:37<00:00, 16.34s/trial, best loss: -0.9494162658031088]


In [15]:
# Train the final model with the best parameters
best_model = XGBClassifier(
    alpha=best_params['alpha'],                    
    subsample=best_params['subsample'],     
    colsample_bytree=best_params['colsample_bytree'],  
    objective='binary:logistic',
    max_depth=int(best_params['max_depth']),             
    min_child_weight=best_params['min_child_weight'],         
    gamma=best_params['gamma'],                
    random_state=42,                 
    n_estimators=int(best_params['n_estimators']),
    learning_rate=best_params['learning_rate']
)

best_model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_test, y_test)],
    eval_metric=lambda y_pred, dmatrix: [('mcc', mcc_metric(y_pred, dmatrix))],
    verbose=True
)

# Evaluate the final model
y_pred = best_model.predict(X_test)
final_score = matthews_corrcoef(y_test, y_pred)
print('Best MCC:', final_score)



/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.59087	validation_0-mcc:0.94942
[1]	validation_0-logloss:0.50738	validation_0-mcc:0.96548
[2]	validation_0-logloss:0.44175	validation_0-mcc:0.97115
[3]	validation_0-logloss:0.38585	validation_0-mcc:0.97252
[4]	validation_0-logloss:0.33962	validation_0-mcc:0.97198
[5]	validation_0-logloss:0.30028	validation_0-mcc:0.97242
[6]	validation_0-logloss:0.26669	validation_0-mcc:0.97254
[7]	validation_0-logloss:0.23885	validation_0-mcc:0.97338
[8]	validation_0-logloss:0.21448	validation_0-mcc:0.97357
[9]	validation_0-logloss:0.19264	validation_0-mcc:0.97453
[10]	validation_0-logloss:0.17402	validation_0-mcc:0.97465
[11]	validation_0-logloss:0.15831	validation_0-mcc:0.97511
[12]	validation_0-logloss:0.14374	validation_0-mcc:0.97565
[13]	validation_0-logloss:0.13108	validation_0-mcc:0.97654
[14]	validation_0-logloss:0.12059	validation_0-mcc:0.97680
[15]	validation_0-logloss:0.11100	validation_0-mcc:0.97723
[16]	validation_0-logloss:0.10256	validation_0-mcc:0.97753
[17]	va

In [16]:
import pickle
with open('../../models/best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# XGB

In [19]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Define the MCC metric for evaluation
def mcc_metric(y_pred, dmatrix):
    y_true = dmatrix.get_label()
    y_pred = (y_pred > 0.5).astype(int)
    mcc = matthews_corrcoef(y_true, y_pred)
    return 'mcc', mcc


In [21]:
# Initialize the XGBClassifier
model = XGBClassifier(                    
    alpha=0.1,                   
    subsample=0.8,     
    colsample_bytree=0.55,  
    objective='binary:logistic',
    max_depth=14,             
    min_child_weight=7,         
    gamma=1e-6,                
    random_state=42,                 
    n_estimators=100             
)

# Train the model
XGB = model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_test, y_test)],
    eval_metric=mcc_metric
)

# Evaluate the model
y_pred = XGB.predict(X_test)
score = matthews_corrcoef(y_test, y_pred)
print('MCC:', score)

# Process the test set
df = pd.read_csv(test_path)

# Ensure the same columns are dropped in the test set
df = df.drop(columns=cols_drop.intersection(df.columns))

# Apply the same ordinal encoding
df[cols_train] = ordinal_encoder.transform(df[cols_train].astype(str))

# Convert all columns to appropriate numeric types
df[cols_train] = df[cols_train].astype(int)

# Drop the 'id' column if it exists in the test set
if 'id' in df.columns:
    df = df.drop(columns=['id'])



/Users/maruanottoni/miniforge3/envs/mlp/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.46945	validation_0-mcc:0.91872
[1]	validation_0-logloss:0.36374	validation_0-mcc:0.95269
[2]	validation_0-logloss:0.27142	validation_0-mcc:0.96986
[3]	validation_0-logloss:0.20504	validation_0-mcc:0.97476
[4]	validation_0-logloss:0.16220	validation_0-mcc:0.97578
[5]	validation_0-logloss:0.13477	validation_0-mcc:0.97682
[6]	validation_0-logloss:0.11449	validation_0-mcc:0.97722
[7]	validation_0-logloss:0.09570	validation_0-mcc:0.97928
[8]	validation_0-logloss:0.08436	validation_0-mcc:0.97978
[9]	validation_0-logloss:0.07728	validation_0-mcc:0.97999
[10]	validation_0-logloss:0.07022	validation_0-mcc:0.98030
[11]	validation_0-logloss:0.06425	validation_0-mcc:0.98040
[12]	validation_0-logloss:0.05759	validation_0-mcc:0.98075
[13]	validation_0-logloss:0.05406	validation_0-mcc:0.98106
[14]	validation_0-logloss:0.04912	validation_0-mcc:0.98184
[15]	validation_0-logloss:0.04575	validation_0-mcc:0.98237
[16]	validation_0-logloss:0.04355	validation_0-mcc:0.98269
[17]	va

In [22]:
# Evaluate the model
y_pred = XGB.predict(X_test)
score = matthews_corrcoef(y_test, y_pred)
print('MCC:', score)

MCC: 0.9836332001545705


# Test set

In [17]:
# Process the test set
df = pd.read_csv(test_path)

# Ensure the same columns are dropped in the test set
df = df.drop(columns=cols_drop.intersection(df.columns))

# Apply the same ordinal encoding
df[cols_train] = ordinal_encoder.transform(df[cols_train].astype(str))

# Convert all columns to appropriate numeric types
df[cols_train] = df[cols_train].astype(int)

# Drop the 'id' column if it exists in the test set
if 'id' in df.columns:
    df = df.drop(columns=['id'])



In [18]:
y_final = best_model.predict(df)

In [19]:
y_final2 = le.inverse_transform(y_final)

In [20]:
sub_sample = pd.read_csv(submission_path)
df_submission = pd.DataFrame({'id': sub_sample['id'], 'class': y_final2})

df_submission.to_csv('../../data/prediction.csv', index=False)
gc.collect()

685

In [21]:
df_submission

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [22]:
sub_sample

,id,class
0,3116945,e
1,3116946,e
2,3116947,e
3,3116948,e
4,3116949,e
...,...,...
2077959,5194904,e
2077960,5194905,e
2077961,5194906,e
2077962,5194907,e
